In [ ]:
from ultralytics import YOLO
import cv2
from ultralytics.utils.plotting import Annotator

In [ ]:
weight = 'C:\\Users\\sorra\\Works\\CS3SEM2\\ML\\system\\models\\best.pt'

model = YOLO(weight)

In [ ]:
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()

    results = model.predict(frame, conf=0.35, stream=True)

    for result in results:

        # https://stackoverflow.com/questions/75324341/yolov8-get-predicted-bounding-box
        annotator = Annotator(frame)

        boxes = result.boxes
        conf_arr = boxes.conf.numpy()
        conf = 0
        for box in boxes:
            b = box.xyxy[0]
            c = box.cls
            
            if len(conf_arr) > 0:
                conf = conf_arr[0]
            annotator.box_label(b, model.names[int(c)]+" "+f'{conf:.2f}')
    
    frame = annotator.result() 

    cv2.imshow('Gun detection', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()